In [26]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import wandb
from wandb.integration.keras import WandbCallback

# Load the data
wandb.login()
data = pd.read_csv("./MIMIC_data.csv")


In [27]:
# data_clean = data.dropna()
# y = data_clean['outcome']
# X = data_clean.drop(columns='outcome')


In [28]:


# build input pipeline using tf.data


def get_train_and_val(data_feaure_and_prediction,BATCH_SIZE = 64):
    """_summary_
    this function gets the relevant feaure and variables 
    Args:
        data_feaure_and_prediction (_type_): the last colum should be the output, the first ones should be the feature(s)
        BATCH_SIZE (int, optional): _description_. Defaults to 64.

    Returns:
        _type_: _description_
    """
    # data_feaure_and_prediction = data_feaure_and_prediction.dropna() # drop the irrelevant values
    X = data_feaure_and_prediction[data_feaure_and_prediction.keys()[:-1]] # the feature
    y = data_feaure_and_prediction[data_feaure_and_prediction.keys()[-1]] # its prediction

    X_standard = StandardScaler()
    X_standard = X_standard.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_standard, y, test_size=0.2, random_state=42)


    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    val_dataset = val_dataset.batch(BATCH_SIZE)
    return [train_dataset, val_dataset]




In [29]:
def make_model(feature_num, input_name="input name"):
    inputs = keras.Input((feature_num,), name=input_name)
    x1 = keras.layers.Dense(64, activation="relu")(inputs)
    x1 = keras.layers.Dropout(0.3)(x1)

    x2 = keras.layers.Dense(64, activation="relu")(x1)
    x2 = keras.layers.Dropout(0.3)(x2)

    outputs = keras.layers.Dense(10, name="predictions")(x2)

    return keras.Model(inputs=inputs, outputs=outputs)



In [30]:
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits) # update the weights of the network

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

def train(train_dataset, val_dataset,  model, optimizer,
          train_acc_metric, val_acc_metric, loss_fn,
          epochs=10, log_step=200, val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_state()
        val_acc_metric.reset_state()

        # ⭐: log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [31]:
# # initialize wandb with your project name and optionally with configutations.
# # play around with the config values and see the result on your wandb dashboard.
# feature = ["age"]

# config = {
#             "epochs": 10,
#             "batch_size": 32,
#             "log_step": 200,
#             "val_log_step": 50,
#             "architecture": "CNN",
#             "dataset": "MIMIC"
#     }

# run = wandb.init(project='my-tf-integration', config=config)
# config = wandb.config

# prediction = "outcome"

# model_data = data[feature+[prediction]].dropna()
# [train_dataset, val_dataset] = get_train_and_val(model_data)


# # Initialize model.
# model = make_model(len(feature))

# # Instantiate an optimizer to train the model.
# optimizer = keras.optimizers.Adam()
# # Instantiate a loss function.
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# # Prepare the metrics.
# train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
# val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


# train(train_dataset,
#     val_dataset, 
#     model,
#     optimizer,
#     train_acc_metric,
#     val_acc_metric,
#     epochs=config.epochs, 
#     log_step=config.log_step, 
#     val_log_step=config.val_log_step)

# run.finish()  # In Jupyter/Colab, let us know you're finished!




In [32]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.

def run_with_feature(feature):
      config = {
                  "epochs": 10,
                  "batch_size": 32,
                  "log_step": 200,
                  "val_log_step": 50,
                  "architecture": "CNN",
                  "dataset": "MIMIC"
            }

      run = wandb.init(project='my-tf-integration', config=config, name=str(feature[0]))
      config = wandb.config

      prediction = "outcome"

      model_data = data[feature+[prediction]].dropna()
      [train_dataset, val_dataset] = get_train_and_val(model_data)


      # Initialize model.
      model = make_model(len(feature))

      # Instantiate an optimizer to train the model.
      optimizer = keras.optimizers.Adam()
      # Instantiate a loss function.
      loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

      # Prepare the metrics.
      train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
      val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


      train(train_dataset,
            val_dataset, 
            model,
            optimizer,
            train_acc_metric,
            val_acc_metric,
            loss_fn,
            epochs=config.epochs, 
            log_step=config.log_step, 
            val_log_step=config.val_log_step)

      run.finish()  # In Jupyter/Colab, let us know you're finished!


feature = ["age"]
run_with_feature(feature)

In [ ]:
# data_keys = data.drop(columns=['outcome']).keys()
# data_keys = data_keys[:30] 

# train_acc_dict = dict.fromkeys(data_keys, 0)
# val_acc_dict = dict.fromkeys(data_keys, 0)

# for elem in data_keys:
#     # try:
#     run_with_feature([elem])
#     #     train_acc_dict[elem] = train_acc_metric
#     #     val_acc_dict[elem] = val_acc_dict
#     # except:
#     #     pass